In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

import sklearn
from sklearn import metrics 
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


/var/folders/jd/f_y1zkqx7qj3nlzj3h0lgsbc0000gn/T/ipykernel_16809/1413956600.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = sm.datasets.fair.load_pandas().data

In [3]:
df.head()

,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affairs
0,3.0,32.0,9.0,3.0,3.0,17.0,2.0,5.0,0.111111
1,3.0,27.0,13.0,3.0,1.0,14.0,3.0,4.0,3.230769
2,4.0,22.0,2.5,0.0,1.0,16.0,3.0,5.0,1.400000
3,4.0,37.0,16.5,4.0,3.0,16.0,5.0,5.0,0.727273
4,5.0,27.0,9.0,1.0,1.0,14.0,3.0,4.0,4.666666


In [4]:
def affair_check(x):
    if x != 0:
        return 1
    else:
        return 0

In [5]:
df['had_affair'] = df['affairs'].apply(affair_check)

In [8]:
df.head()

,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affairs,had_affair
0,3.0,32.0,9.0,3.0,3.0,17.0,2.0,5.0,0.111111,1
1,3.0,27.0,13.0,3.0,1.0,14.0,3.0,4.0,3.230769,1
2,4.0,22.0,2.5,0.0,1.0,16.0,3.0,5.0,1.400000,1
3,4.0,37.0,16.5,4.0,3.0,16.0,5.0,5.0,0.727273,1
4,5.0,27.0,9.0,1.0,1.0,14.0,3.0,4.0,4.666666,1


In [7]:
df['had_affair'].value_counts()

had_affair
0    4313
1    2053
Name: count, dtype: int64

In [9]:
df.groupby('had_affair').mean()

,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affairs
had_affair,,,,,,,,,
0,4.329701,28.390679,7.989335,1.238813,2.504521,14.322977,3.405286,3.833758,0.000000
1,3.647345,30.537019,11.152460,1.728933,2.261568,13.972236,3.463712,3.884559,2.187243


In [14]:
occ_dummies = pd.get_dummies(df['occupation']).astype(int)
hus_occ_dummies = pd.get_dummies(df['occupation_husb']).astype(int)

In [15]:
# Create column names for the new DataFrames
occ_dummies.columns = ['occ1' ,'occ2', 'occ3', 'occ4', 'occ5', 'occ6']
hus_occ_dummies.columns = ['hocc1' ,'hocc2', 'hocc3', 'hocc4', 'hocc5', 'hocc6']

In [24]:
# Set X as new DataFrame without the occupation columns or the Y target

In [16]:
# Concat the dummy DataFrames Together
dummies = pd.concat([occ_dummies, hus_occ_dummies], axis=1)

In [17]:
dummies.head()

,occ1,occ2,occ3,occ4,occ5,occ6,hocc1,hocc2,hocc3,hocc4,hocc5,hocc6
0,0,1,0,0,0,0,0,0,0,0,1,0
1,0,0,1,0,0,0,0,0,0,1,0,0
2,0,0,1,0,0,0,0,0,0,0,1,0
3,0,0,0,0,1,0,0,0,0,0,1,0
4,0,0,1,0,0,0,0,0,0,1,0,0


In [19]:
# Now Concat the X DataFrame with the dummy variables
X = df.drop(['occupation', 'occupation_husb', 'had_affair'],axis=1)

X = pd.concat([X, dummies],axis=1)

X.head()



,rate_marriage,age,yrs_married,children,religious,educ,affairs,occ1,occ2,occ3,occ4,occ5,occ6,hocc1,hocc2,hocc3,hocc4,hocc5,hocc6
0,3.0,32.0,9.0,3.0,3.0,17.0,0.111111,0,1,0,0,0,0,0,0,0,0,1,0
1,3.0,27.0,13.0,3.0,1.0,14.0,3.230769,0,0,1,0,0,0,0,0,0,1,0,0
2,4.0,22.0,2.5,0.0,1.0,16.0,1.400000,0,0,1,0,0,0,0,0,0,0,1,0
3,4.0,37.0,16.5,4.0,3.0,16.0,0.727273,0,0,0,0,1,0,0,0,0,0,1,0
4,5.0,27.0,9.0,1.0,1.0,14.0,4.666666,0,0,1,0,0,0,0,0,0,1,0,0


In [25]:
# Set Y as Target class, Had Affair

y = df['had_affair']

In [26]:
X.corr()

,rate_marriage,age,yrs_married,children,religious,educ,occ2,occ3,occ4,occ5,occ6,hocc2,hocc3,hocc4,hocc5,hocc6
rate_marriage,1.000000,-0.111127,-0.128978,-0.129161,0.078794,0.079869,-0.019697,-0.053082,0.068882,-0.002109,0.008878,-0.038992,-0.022514,0.003303,0.003256,0.039561
age,-0.111127,1.000000,0.894082,0.673902,0.136598,0.027960,-0.034223,-0.066371,0.040982,0.079533,0.030676,-0.057368,0.011610,-0.048989,0.105525,0.083212
yrs_married,-0.128978,0.894082,1.000000,0.772806,0.132683,-0.109058,0.004668,-0.021261,-0.026816,0.076820,-0.004912,-0.033451,0.008046,-0.031121,0.092462,0.042921
children,-0.129161,0.673902,0.772806,1.000000,0.141845,-0.141918,0.081182,-0.063298,-0.003235,0.033274,-0.026830,0.001190,-0.005538,-0.008032,0.053965,0.024260
religious,0.078794,0.136598,0.132683,0.141845,1.000000,0.032245,-0.013129,-0.034986,0.043996,0.004260,0.011784,0.009990,0.008170,-0.008491,-0.000063,0.006558
educ,0.079869,0.027960,-0.109058,-0.141918,0.032245,1.000000,-0.217719,-0.335615,0.477505,-0.022121,0.226920,-0.160756,-0.052723,-0.031422,0.042540,0.223167
occ2,-0.019697,-0.034223,0.004668,0.081182,-0.013129,-0.217719,1.000000,-0.348075,-0.251243,-0.143237,-0.052128,0.183782,-0.020904,-0.009786,-0.093292,-0.059107
occ3,-0.053082,-0.066371,-0.021261,-0.063298,-0.034986,-0.335615,-0.348075,1.000000,-0.560645,-0.319631,-0.116322,-0.000638,0.090043,0.011248,0.003021,-0.101673
occ4,0.068882,0.040982,-0.026816,-0.003235,0.043996,0.477505,-0.251243,-0.560645,1.000000,-0.230712,-0.083962,-0.083123,-0.043159,0.037341,-0.001946,0.085766
occ5,-0.002109,0.079533,0.076820,0.033274,0.004260,-0.022121,-0.143237,-0.319631,-0.230712,1.000000,-0.047868,-0.053426,-0.044053,-0.039932,0.114903,0.006016


In [23]:
# Dropping one column of each dummy variable set to avoid multicollinearity

#X.drop('occ1',axis=1, inplace=True)
#X.drop('hocc1',axis=1, inplace=True)


# Drop affairs column so Y target makes sense

#X.drop('affairs',axis=1,inplace=True)


In [7]:
# This adds a column of 1's to the dataframe. 
# The model will not run without, but if 
# it could every model would try to pass through the origin


In [29]:
logit_model = sm.Logit(y, sm.add_constant(X))
results = logit_model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.542911
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             had_affair   No. Observations:                 6366
Model:                          Logit   Df Residuals:                     6349
Method:                           MLE   Df Model:                           16
Date:                Wed, 14 Feb 2024   Pseudo R-squ.:                  0.1365
Time:                        17:37:23   Log-Likelihood:                -3456.2
converged:                       True   LL-Null:                       -4002.5
Covariance Type:            nonrobust   LLR p-value:                1.534e-222
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const             2.9708      0.572      5.192      0.000       1.849       4.092
rate_marriage    -0.7102      0.031    -22.560      0.000      -0.772      -0.649
age              -0.0613      0.010     -5.936      0.000      -0.082      -0.041
yrs_married       0.1080      0.011      9.836      0.000       0.086       0.129
children          0.0156      0.032      0.488      0.625      -0.047       0.078
religious        -0.3754      0.035    -10.766      0.000      -0.444      -0.307
educ             -0.0017      0.017     -0.099      0.921      -0.036       0.032
occ2              0.3902      0.448      0.872      0.383      -0.487       1.267
occ3              0.7027      0.441      1.592      0.111      -0.163       1.568
occ4              0.4714      0.443      1.065      0.287      -0.396       1.339
occ5              1.0542      0.447      2.360      0.018       0.179       1.930
occ6              1.1080      0.494      2.242      0.025       0.139       2.077
hocc2             0.1704      0.186      0.916      0.360      -0.194       0.535
hocc3             0.2842      0.202      1.406      0.160      -0.112       0.680
hocc4             0.1428      0.181      0.789      0.430      -0.212       0.498
hocc5             0.1723      0.183      0.944      0.345      -0.186       0.530
hocc6             0.1828      0.204      0.897      0.369      -0.216       0.582
=================================================================================
"""

In [30]:
X.drop(columns=['children','educ','occ2','occ3','occ4',
'hocc2','hocc3','hocc4','hocc5','hocc6'],inplace=True)

In [34]:
#result of preliminary run
logit_model = sm.Logit(y, sm.add_constant(X)).fit()
logit_model.summary()

Optimization terminated successfully.
         Current function value: 0.544657
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             had_affair   No. Observations:                 6366
Model:                          Logit   Df Residuals:                     6359
Method:                           MLE   Df Model:                            6
Date:                Wed, 14 Feb 2024   Pseudo R-squ.:                  0.1337
Time:                        17:47:09   Log-Likelihood:                -3467.3
converged:                       True   LL-Null:                       -4002.5
Covariance Type:            nonrobust   LLR p-value:                5.057e-228
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const             3.7516      0.262     14.317      0.000       3.238       4.265
rate_marriage    -0.7166      0.031    -22.880      0.000      -0.778      -0.655
age              -0.0632      0.010     -6.384      0.000      -0.083      -0.044
yrs_married       0.1127      0.009     12.028      0.000       0.094       0.131
religious        -0.3750      0.035    -10.831      0.000      -0.443      -0.307
occ5              0.4731      0.088      5.398      0.000       0.301       0.645
occ6              0.5271      0.222      2.377      0.017       0.093       0.962
=================================================================================
"""

In [38]:
preds = results.pred_table()

confusion = pd.DataFrame(preds, index = ['Did Not', 'Did'], columns=['Pred Did Not', 'Pred Did'])
confusion

,Pred Did Not,Pred Did
Did Not,3883.0,430.0
Did,1318.0,735.0


In [ ]:
TPR=(float(TP) / (TP + FN))
TPN=(float(TN) / (TN + FP)) 
PPV=(float(TP) / (TP + FP)) 
NPV=(float(TN) / (TN + FN)) 
FNR=(float(FN) / (FN + TP))
FPR=(float(FP) / (FP + TN))
FDR=(float(FP) / (FP + TP))
FOR=(float(FN) / (FN + TN))
TS=(float(TP) / (TP+FN + FP))
ACC=(float(TP+TN) / (TP+FP+FN + TN))

print (f"sensitivity, recall, hit rate, or true positive rate (TPR): {TPR:.3f}")
print (f"specificity, selectivity or true negative rate (TNR): {TPN:.3f}")
print (f"precision or positive predictive value (PPV): {PPV:.3f}")
print (f"negative predictive value (NPV): {NPV:.3f}")
print (f"miss rate or false negative rate (FNR): {FNR:.3f}")
print (f"fall-out or false positive rate (FPR): {FPR:.3f}")
print (f"false discovery rate (FDR): {FDR:.3f}")
print (f"false omission rate (FOR): {FOR:.3f}")
print (f"Threat score (TS) or Critical Success Index (CSI):{TS:.3f} ")
print("")
print (f"accuracy (ACC): {ACC:.3f}")
print (f"F1: {F1:.3f}")
